This notebook attempts to find correlation between mean feature values and mean response. The idea is that mean y corresponds to market returns and if we can predict market returns and dispersion around it we could possibly predict the returns of each id. Just an idea at the moment and needs more work.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
with pd.HDFStore("../input/train.h5", "r") as train:
    # Note that the "train" dataframe is the only dataframe in the file
    df = train.get("train")

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
col = 'fundamental_7'

In [ ]:
groupbyId = df.groupby('id')
tm = groupbyId[['technical_20']].mean().values[:, 0]
ts = groupbyId[['technical_20']].std().values[:, 0]
ym = groupbyId[['y']].mean().values[:, 0]
ys = groupbyId[['y']].std().values[:, 0]
ys[ys==0] = 1.
ts[ts==0] = 1.
tm /= ts
ym /= ys
np.isnan(tm).any()

In [ ]:
from sklearn import preprocessing

print(np.correlate(tm, ym))

In [ ]:
import kagglegym
env = kagglegym.make()
observation = env.reset()
train = observation.train

In [ ]:
import statsmodels.api as smapi

train.fillna(0, inplace=True)

rsquared = {}

for col in train.columns:
    groupbyId = train.groupby('id')
    tm = groupbyId[[col]].mean().values[:, 0]
    ts = groupbyId[[col]].std().values[:, 0]
    ym = groupbyId[['y']].mean().values[:, 0]
    ys = groupbyId[['y']].std().values[:, 0]
    ys[ys==0] = 1.
    ts[ts==0] = 1.
    tm = tm / ts
    ym = ym / ys
    model = smapi.OLS(ym, smapi.add_constant(tm)).fit()
    rsquared[col] = model.rsquared_adj
print(model.summary())